# Loading real data

In [0]:
#!kill -9 -1

In [0]:
import random
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.metrics
import scipy as sp

In [0]:
"""
Created on Tue Nov  6 10:06:52 2018

@author: yandexdataschool

Original Code found in:
https://github.com/yandexdataschool/roc_comparison

updated: Raul Sanchez-Vazquez
"""

import scipy.stats
from scipy import stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)


def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [8]:
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num','marital-status', 'occupation', 'relationship', 'race', 'gender','capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income']
import pandas as pd
url_train = 'drive/My Drive/Discrétisation ICLR19/opendata/adult.data'
url_test = 'drive/My Drive/Discrétisation ICLR19/opendata/adult.test'
#url_train = '~/Google Drive/Discrétisation ICLR19/opendata/adult.data'
#url_test = '~/Google Drive/Discrétisation ICLR19/opendata/adult.test'
train = pd.read_csv(url_train, sep=",\s", header=None, names = column_names, engine = 'python')
train
test = pd.read_csv(url_test, sep=",\s", header=None, names = column_names, engine = 'python', skiprows=1)
test['income'].replace(regex=True,inplace=True,to_replace=r'\.',value=r'')
adult = pd.concat([test,train])

FileNotFoundError: ignored

In [6]:
adult.income = sklearn.preprocessing.LabelEncoder().fit_transform(adult.income)

adult.drop(['fnlwgt','educational-num'],inplace=True, axis=1)

NameError: ignored

In [7]:
adult.reset_index(inplace=True, drop=True)

NameError: ignored

# Establishing 1st benchmark: naïve logistic regression

## Logistic Regression

### Label Encoding

In [0]:
adult_label_encoders = []

adult_encoded = adult.copy()

for j in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender']:
    temp = sk.preprocessing.LabelEncoder()
    temp.fit(adult[j].astype(str))
    adult_label_encoders.append(temp)
    adult_encoded[j] = temp.transform(adult[j].astype(str))

### One-hot encoding

In [0]:
#!pip install --upgrade scikit-learn

In [0]:
adult_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
adult_one_hot_encoder.fit(adult_encoded[[
    'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender'
]])
adult_one_hot_encoded = adult_encoded.copy()
adult_one_hot_encoded.drop(
    ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender'],
    axis=1,
    inplace=True)
adult_one_hot_encoded = pd.concat(
    [
        adult_one_hot_encoded,
        pd.DataFrame(
            adult_one_hot_encoder.transform(adult_encoded[[
                'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender'
            ]]),
            index=adult_one_hot_encoded.index)
    ],
    axis=1)

### Data split

In [0]:
import sklearn.model_selection

In [0]:
adult_features_train, adult_features_test, adult_perf_train, adult_perf_test = sk.model_selection.train_test_split(
    adult_one_hot_encoded.drop('income', axis=1),
    adult_one_hot_encoded.income,
    test_size=0.33,
    random_state=1)

In [0]:
adult_nn_features_train = adult_encoded.iloc[
    adult_features_train.index, :].drop(
        'income', axis=1)
adult_nn_features_test = adult_encoded.iloc[adult_features_test.index, :].drop(
    'income', axis=1)
adult_nn_perf_train = adult_encoded.iloc[
    adult_features_train.index, :].income
adult_nn_perf_test = adult_encoded.iloc[adult_features_test.index, :].income

### LR on train data

In [0]:
import sklearn.linear_model

In [0]:
adult_naive_LR = sk.linear_model.LogisticRegression(C=1e20, tol=1e-8, solver="newton-cg")
adult_naive_LR.fit(
    adult_features_train[adult_features_train.isna().sum(axis=1) == 0],
    adult_perf_train[adult_features_train.isna().sum(axis=1) == 0])

/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did

LogisticRegression(C=1e+20, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=1e-08, verbose=0, warm_start=False)

### Application of learnt LR on test data

In [0]:
AUC_LR = sk.metrics.roc_auc_score(
    adult_perf_test,
    adult_naive_LR.predict_proba(adult_features_test)[:, 1])
2 * AUC_LR - 1

0.8150530331027663

In [0]:
alpha = .95
y_pred = adult_naive_LR.predict_proba(adult_features_test)[:, 1]
y_true = adult_perf_test

auc, auc_cov = delong_roc_variance(
    y_true,
    y_pred)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('Gini:', 2*auc-1)
print('AUC COV:', auc_cov)
print('95% Gini CI:', 2*ci-1)


Gini: 0.8150530331027666
AUC COV: 6.096126731962454e-06
95% Gini CI: [0.8053746  0.82473147]


In [0]:
#import math

#Q1 = AUC_LR / (2-AUC_LR)

#Q2 = 2*AUC_LR**2 / (1+AUC_LR)

#SE_AUC = math.sqrt((AUC_LR*(1-AUC_LR) + (sum(adult_perf_test)-1)*(Q1-AUC_LR**2) + ((adult_perf_test.shape[0]-sum(adult_perf_test))-1)*(Q2-AUC_LR**2))/((adult_perf_test.shape[0]-sum(adult_perf_test))*sum(adult_perf_test)))

#AUC_LR + 1.96*SE_AUC
#AUC_LR - 1.96*SE_AUC

#2*(AUC_LR + 1.96*SE_AUC)-1

In [0]:
#2*(AUC_LR - 1.96*SE_AUC)-1

In [0]:
#2*1.96*SE_AUC*100

# Establishing 2nd benchmark: MDLP disc + Chi2 grouping

## MDLP disc

In [0]:
#!pip install mdlp-discretization
from mdlp.discretization import MDLP
transformer_cont_adult = MDLP()

In [0]:
transformer_cont_adult.fit(
    adult_nn_features_train[[
        'age','capital-gain','capital-loss','hours-per-week'
    ]], adult_nn_perf_train)

MDLP(continuous_features=None, min_depth=0, random_state=None, shuffle=True)

In [0]:
#np.unique(transformer_cont_adult.transform(
#    adult_nn_features_test[[
#        'age','capital-gain','capital-loss','hours-per-week'
#    ]])[:,2])

In [0]:
score_adult_MDLP_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
score_adult_MDLP_one_hot_encoder.fit(
    transformer_cont_adult.transform(adult_nn_features_train[[
        'age','capital-gain','capital-loss','hours-per-week'
    ]]))

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=False)

## Grouping

In [0]:
def chi2_test(liste):
    try:
        return sp.stats.chi2_contingency(liste)[1]
    except Exception:
        return 1

In [0]:
adult_train_grouped = adult.iloc[adult_features_train.index, :].copy()
d = dict((x, []) for x in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender'])

for var in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender']:
    
    adult_train_grouped[var] = adult_train_grouped[var].astype(str)
    d[var] = [x for x in np.unique(adult_train_grouped[var])]
    p_value = 1

    while(p_value>0.05):
        freq_table = adult_train_grouped.groupby([var,'income']).size().reset_index()
        liste_paires_modalites = [[a,b] for a in np.unique(adult_train_grouped[var]) for b in np.delete(np.unique(adult_train_grouped[var]),np.where(np.unique(adult_train_grouped[var])==a))]
        liste_chi2 = [chi2_test([freq_table.iloc[np.in1d(freq_table[var],pair[0]),2],freq_table.iloc[np.in1d(freq_table[var],pair[1]),2]]) for pair in liste_paires_modalites]
        p_value = max(liste_chi2)

        if (p_value>0.05 and len(np.unique(adult_train_grouped[var]))>1):
            adult_train_grouped[var].iloc[np.in1d(adult_train_grouped[var],liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))])] = liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][0] + ' - ' + liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][1]
            d[var].remove(liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][0])
            d[var].remove(liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][1])
            d[var].append(str(liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][0] + ' - ' + liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][1]))
            print('Feature '+var+ ' - levels merged : '+str(liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))]))
        else: break


Feature workclass - levels merged : ['?', 'Never-worked']
Feature workclass - levels merged : ['Local-gov', 'Self-emp-not-inc']
Feature workclass - levels merged : ['Private', 'Without-pay']
Feature workclass - levels merged : ['Local-gov - Self-emp-not-inc', 'State-gov']
Feature education - levels merged : ['10th', '5th-6th']
Feature education - levels merged : ['10th - 5th-6th', '7th-8th']
Feature education - levels merged : ['Doctorate', 'Prof-school']
Feature education - levels merged : ['1st-4th', 'Preschool']
Feature education - levels merged : ['11th', '9th']
Feature education - levels merged : ['10th - 5th-6th - 7th-8th', '12th']
Feature education - levels merged : ['Assoc-acdm', 'Assoc-voc']
Feature education - levels merged : ['11th - 9th', '10th - 5th-6th - 7th-8th - 12th']
Feature education - levels merged : ['11th - 9th - 10th - 5th-6th - 7th-8th - 12th', '1st-4th - Preschool']
Feature marital-status - levels merged : ['Divorced', 'Married-spouse-absent']
Feature marital-s

## Test time

In [0]:
adult_train_mdlp = score_adult_MDLP_one_hot_encoder.transform(
    transformer_cont_adult.transform(adult_nn_features_train[[
        'age','capital-gain','capital-loss','hours-per-week'
    ]]))

In [0]:
adult_train_grouped_label_encoders = []

adult_train_grouped_encoded = adult_train_grouped.copy()

for j in [
        'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender'
]:
    temp = sk.preprocessing.LabelEncoder()
    temp.fit(adult_train_grouped_encoded[j].astype(str))
    adult_train_grouped_label_encoders.append(temp)
    adult_train_grouped_encoded[j] = temp.transform(adult_train_grouped_encoded[j].astype(str))

In [0]:
score_adult_CHI2_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")

score_adult_CHI2_one_hot_encoder.fit(
        adult_train_grouped_encoded[[
            'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender'
        ]])

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=False)

In [0]:
adult_train_chi2 = score_adult_CHI2_one_hot_encoder.transform(adult_train_grouped_encoded[[
            'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender'
        ]])

In [0]:
adult_adhoc_train = np.concatenate((adult_train_chi2,adult_train_mdlp),axis=1)

In [0]:
adult_adhoc_LR = sk.linear_model.LogisticRegression(C=1e20, tol=1e-8, solver="newton-cg")
adult_adhoc_LR.fit(
    adult_adhoc_train,
    adult_train_grouped_encoded['income'])

LogisticRegression(C=1e+20, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=1e-08, verbose=0, warm_start=False)

In [0]:
adult_test_grouped = adult.iloc[adult_features_test.index, :].copy()

for var in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender']:
    
    adult_test_grouped[var] = adult_test_grouped[var].astype(str)

    for x in d[var]:
        if x.find(' - ')>-1:
            liste_modalites = x.split(' - ')
            adult_test_grouped[var].iloc[np.in1d(adult_test_grouped[var],liste_modalites)] = x
        

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
adult_test_grouped_encoded = adult_test_grouped.copy()

for j in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender']:
    indice = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender'].index(j)
    adult_test_grouped_encoded[j] = adult_train_grouped_label_encoders[indice].transform(adult_test_grouped_encoded[j].astype(str))

In [0]:
adult_test_chi2 = score_adult_CHI2_one_hot_encoder.transform(adult_test_grouped_encoded[[
            'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country', 'race','gender'
        ]])

In [0]:
adult_adhoc_test = np.concatenate((adult_test_chi2,adult_test_mdlp),axis=1)

In [0]:
AUC_LR = sk.metrics.roc_auc_score(
    adult_perf_test,
    adult_adhoc_LR.predict_proba(adult_adhoc_test)[:, 1])
2 * AUC_LR - 1

0.8530011392408037

In [0]:
alpha = .95
y_pred = adult_adhoc_LR.predict_proba(adult_adhoc_test)[:, 1]
y_true = adult.iloc[adult_features_test.index, :]['income']

auc, auc_cov = delong_roc_variance(
    y_true,
    y_pred)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('Gini:', 2*auc-1)
print('AUC COV:', auc_cov)
print('95% Gini CI:', 2*ci-1)

Gini: 0.8530011392408037
AUC COV: 4.693540700382635e-06
95% Gini CI: [0.84450878 0.8614935 ]


In [0]:
#AUC_adhoc = sk.metrics.roc_auc_score(
#    adult_grouped_encoded.iloc[adult_features_test.index, :]
#              ['income'],
#    adult_adhoc_LR.predict_proba(adult_adhoc_test)[:, 1])
#2 * AUC_adhoc - 1

In [0]:
#Q1 = AUC_adhoc / (2-AUC_adhoc)

#Q2 = 2*AUC_adhoc**2 / (1+AUC_adhoc)

#SE_AUC = math.sqrt((AUC_adhoc*(1-AUC_adhoc) + (sum(adult_perf_test)-1)*(Q1-AUC_adhoc**2) + ((adult_perf_test.shape[0]-sum(adult_perf_test))-1)*(Q2-AUC_adhoc**2))/((adult_perf_test.shape[0]-sum(adult_perf_test))*sum(adult_perf_test)))

#2 * AUC_adhoc - 1 + 2*1.96*SE_AUC

In [0]:
def hyperparam_opt_choice():
  
    #opt_choice = {{choice([optimizers.SGD, optimizers.RMSprop, optimizers.Adagrad, optimizers.Adadelta, optimizers.Adam, optimizers.Adamax, optimizers.Nadam])}}
    opt_choice = {{choice([optimizers.SGD, optimizers.RMSprop])}}

    if opt_choice == optimizers.SGD:
        lr_sgd = {{uniform(0.001,2)}}
        momentum_sgd = {{uniform(0,0.5)}}
        decay_sgd = {{uniform(0,0.001)}}
        nesterov_sgd = {{choice([True, False])}}
        opt = opt_choice(lr_sgd,momentum_sgd,decay_sgd,nesterov_sgd)
    elif opt_choice == optimizers.RMSprop:
        lr_RMS = {{uniform(0.001,2)}}
        decay_RMS = {{uniform(0,0.001)}}
        rho_RMS = {{uniform(0.7,1)}}
        epsilon_RMS = None
        opt = opt_choice(lr_RMS,decay_RMS,rho_RMS,epsilon_RMS)
    elif opt_choice == optimizers.Adagrad:
        lr_adagrad = {{}}
        epsilon_adagrad = {{}}
        decay_adagrad = {{}}
        opt = opt_choice(lr_adagrad,epsilon_adagrad,decay_adagrad)
    elif opt_choice == optimizers.Adadelta:
        lr_adadelta = {{}}
        rho_adadelta = {{}}
        epsilon_adadelta = {{}}
        decay_adadelta = {{}}
        opt = opt_choice(lr_adadelta,rho_adadelta,epsilon_adadelta,decay_adadelta)
    elif opt_choice == optimizers.Adam:
        lr_adam = {{}}
        beta_1_adam = {{}}
        beta_2_adam = {{}}
        epsilon_adam = {{}}
        decay_adam = {{}}
        amsgrad_adam = {{}}
        opt = opt_choice(lr_adam,beta_1_adam,beta_2_adam,epsilon_adam,decay_adam,amsgrad_adam)
    elif opt_choice == optimizers.Adamax:
        lr_adamax = {{}}
        beta_1_adamax = {{}}
        beta_2_adamax = {{}}
        epsilon_adamax = {{}}
        decay_adamax = {{}}
        opt = opt_choice(lr_adamax,beta_1_adamax,beta_2_adamax,epsilon_adamax,decay_adamax)
    elif opt_choice == optimizers.Nadam:
        lr_nadam = {{}}
        beta_1_nadam = {{}}
        beta_2_nadam = {{}}
        epsilon_nadam = {{}}
        schedule_decay_nadam = {{}}
        opt = opt_choice(lr_nadam,beta_1_nadam,beta_2_nadam,schedule_decay_nadam)
      
    return (opt)
